# Appendix 7 - Gender Inference - Results Testing

Note: When running the rule based gender classification algorithm on the dataset, we used the version of the raw, unprocessed dataset, as slightly different pre-processing was required to get accurate results from the gender algorithm (tokenisation of the description field, lowercase text).

The final rule based algorithm classified our full dataset as follows:

In [1]:
import pandas as pd

# Import the raw dataset
tweets = pd.read_excel("01sotu_final_corpus.xlsx")

print(tweets.head)

<bound method NDFrame.head of                                   date  \
0       Wed Jan 31 03:30:46 +0000 2018   
1       Wed Jan 31 03:30:46 +0000 2018   
2       Wed Jan 31 03:30:46 +0000 2018   
3       Wed Jan 31 03:30:46 +0000 2018   
4       Wed Jan 31 03:30:46 +0000 2018   
5       Wed Jan 31 03:30:46 +0000 2018   
6       Wed Jan 31 03:30:46 +0000 2018   
7       Wed Jan 31 03:30:46 +0000 2018   
8       Wed Jan 31 03:30:46 +0000 2018   
9       Wed Jan 31 03:30:46 +0000 2018   
10      Wed Jan 31 03:30:46 +0000 2018   
11      Wed Jan 31 03:30:46 +0000 2018   
12      Wed Jan 31 03:30:46 +0000 2018   
13      Wed Jan 31 03:30:46 +0000 2018   
14      Wed Jan 31 03:30:46 +0000 2018   
15      Wed Jan 31 03:30:46 +0000 2018   
16      Wed Jan 31 03:30:46 +0000 2018   
17      Wed Jan 31 03:30:46 +0000 2018   
18      Wed Jan 31 03:30:46 +0000 2018   
19      Wed Jan 31 03:30:46 +0000 2018   
20      Wed Jan 31 03:30:46 +0000 2018   
21      Wed Jan 31 03:30:46 +0000 2018   
22  

In [2]:
#PREPARATION OF NAME DATA

# Import dataset of names and their genders
namesfinal = pd.read_excel("top2000names2014final.xlsx")

# Sort names dataframe so longest names come first (so that Erica will then match against Erica and not Eric)
namesfinal["namelength"] = ""
for i in range(len(namesfinal)):
    namesfinal.at[i, "namelength"] = len(namesfinal.at[i, "name"])
namesfinal = namesfinal.sort_values(by=['namelength'], ascending=False)
namesfinal = namesfinal.reset_index(drop=True)
print(namesfinal.shape)

# Remove any names 3 characters or less
namesfinal.drop(namesfinal[namesfinal.namelength < 4].index, inplace=True)
print(namesfinal.shape)

# Create a column with name converted to lowercase
namesfinal["name_lower"] = namesfinal["name"].str.lower()

(2013, 4)
(1944, 4)


In [3]:
#ASSIGNMENT OF GENDER BASED ON USERNAME

# Create a column with username converted to lowercase
tweets["user_lower"] = tweets["user"].str.lower()

# Create 2 columns for the algorithm to make its gender prediction in
tweets["gender_username_algo"] = ""
tweets["namematched_algo"] = ""

# Run the name matching algorithm
for i in range(len(tweets)):
    for j in range (len(namesfinal)):
        if tweets.at[i, "gender_username_algo"]=="":
            if (namesfinal.at[j, "name_lower"] in tweets.at[i, "user_lower"]):
                tweets.at[i, "gender_username_algo"] = namesfinal.at[j, "gender"]
                tweets.at[i, "namematched_algo"] = namesfinal.at[j, "name"]
            else:
                continue
        else:
            continue
    # Just to get a sense of progress while processing
    if i%1000==0:
        (print(i))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [4]:
#ASSIGNMENT OF GENDER BASED ON DESCRIPTION

descr_keywords = pd.read_excel("descr_keywords.xlsx")

# Add column for new gender prediction based on description
# Create a column with username converted to lowercase, on both datasets
tweets["gender_description_algo"] = ""

# Make description field lowercase
tweets["description"] = tweets["description"].str.lower()

# Tokenize the description field using natural language toolkit
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

tweets["description_tokenized"] = ""
tokenizer = RegexpTokenizer(r'\w+')

# Run the tokenizing algorithm
for i in range(len(tweets)):
    if pd.notnull(tweets.at[i, 'description'])==False :
        (tweets.at[i, 'description_tokenized'])=""
    else:
        (tweets.at[i, 'description_tokenized'])=tokenizer.tokenize((tweets.at[i, 'description']))

descr_keywords.columns = ['word', 'gender']

# Run the gender assigning algorithm
for i in range(len(tweets)):
    for j in range(len(descr_keywords)):
        if (descr_keywords.at[j, 'word'] in tweets.at[i, 'description_tokenized']) & (tweets.at[i, 'gender_description_algo']==""):
            tweets.at[i, 'gender_description_algo'] = descr_keywords.at[j, 'gender']
        else:
            continue

In [5]:
# ASSIGNMENT OF FINAL GENDER VALUE

tweets["gender_final"] = ""

for i in range(len(tweets)):
    tweets.at[i, "gender_final"] = tweets.at[i, "gender_description_algo"] 
    if tweets.at[i, "gender_final"]=="":
        tweets.at[i, "gender_final"]=tweets.at[i, "gender_username_algo"]

In [6]:
# Export the tweet subset to excel for checking
writer = pd.ExcelWriter('04sotu_with_gender.xlsx')
tweets.to_excel(writer,'Sheet1')
writer.save()

In [7]:
# Check how many genders have been assigned
gendercount=0
for i in range(len(tweets)):
        if tweets.at[i, 'gender_final'] !="":
            gendercount+=1
print(gendercount)

138936


In [8]:
print(tweets.shape)

(274520, 17)


In [9]:
138936/274520

0.5061052018067901

The gender algorithm assigned 138,936 rows with a gender, out of a total 274,520, giving 51% of our dataset a gender classification.

#### Evaluation of accuracy

In [10]:
# Export a random sample of 1800 rows to be checked for accuracy of classifications
tweets_sample=tweets.sample(1800)

writer = pd.ExcelWriter('04sotu_with_gender_sample.xlsx')
tweets_sample.to_excel(writer,'Sheet1')
writer.save()

In [15]:
import pandas as pd
# Reimport the checked sample
tweets_sample1000_withgender_checked = pd.read_excel('04sotu_with_gender_sample.xlsx')


In [16]:
# Pivots to view aggregated results
pivot0=pd.pivot_table(tweets_sample1000_withgender_checked, index=["gender_final"],values=["user"], aggfunc=[len], margins=True)
pivot0.columns=['count']
print(pivot0, '\n')

pivot1=pd.pivot_table(tweets_sample1000_withgender_checked, index=["humancheck"],values=["user"], aggfunc=[len], margins=True)
pivot1.columns=['count']
print(pivot1, '\n')

              count
gender_final       
F               389
M               536
U               875
All            1800 

            count
humancheck       
OK            801
notsure        80
wrong          44
All           925 



The first pivot shows us that the algorithm assigned gender to 925/1800 rows (of which, incidentally: 389 F, 536 M)

The second pivot shows us that we have judged 801 (86.6%) of these assignments as 'OK' (the human agrees with the gender assignment), 80 (8.6%) as 'notsure' (the human finds the user's gender ambiguous) and 44 (4.8%) as wrong (the human is sure the gender is in fact the opposite of that assigned by the algorithm)

This gives an accuracy rate of 86.6 - 95.2 %

The final step was to copy the column of gender values to the version of the dataset where preprocessing had been carried out. As the two datasets are identical in terms of row number and order of tweets ( as checked below) this was achieved via a single loop:

In [22]:
print(tweets.shape)
print(tweets_final.shape)

(274520, 17)
(274520, 19)


In [18]:
# Copy the gender column into the pre-processed dataset
tweets_final = pd.read_excel("03sotu_with_states.xlsx")

print(tweets_final.head)

<bound method NDFrame.head of                                   date  \
0       Wed Jan 31 03:30:46 +0000 2018   
1       Wed Jan 31 03:30:46 +0000 2018   
2       Wed Jan 31 03:30:46 +0000 2018   
3       Wed Jan 31 03:30:46 +0000 2018   
4       Wed Jan 31 03:30:46 +0000 2018   
5       Wed Jan 31 03:30:46 +0000 2018   
6       Wed Jan 31 03:30:46 +0000 2018   
7       Wed Jan 31 03:30:46 +0000 2018   
8       Wed Jan 31 03:30:46 +0000 2018   
9       Wed Jan 31 03:30:46 +0000 2018   
10      Wed Jan 31 03:30:46 +0000 2018   
11      Wed Jan 31 03:30:46 +0000 2018   
12      Wed Jan 31 03:30:46 +0000 2018   
13      Wed Jan 31 03:30:46 +0000 2018   
14      Wed Jan 31 03:30:46 +0000 2018   
15      Wed Jan 31 03:30:46 +0000 2018   
16      Wed Jan 31 03:30:46 +0000 2018   
17      Wed Jan 31 03:30:46 +0000 2018   
18      Wed Jan 31 03:30:46 +0000 2018   
19      Wed Jan 31 03:30:46 +0000 2018   
20      Wed Jan 31 03:30:46 +0000 2018   
21      Wed Jan 31 03:30:46 +0000 2018   
22  

In [24]:
# Check the ordering of the datasets is identical by matching on id string:

tweets_final["gender"] = ""

count=0;
for i in range(len(tweets_final)):
    if tweets_final.at[i, "id_str"] == tweets.at[i, "id_str"]:
        count+=1
print(count)

274520


In [26]:
# The above shows that the order of rows in both datasets is identical, and the gender values can be copied across using the above method
for i in range(len(tweets_final)):
    tweets_final.at[i, "gender"] = tweets.at[i, "gender_final"]


In [29]:
# Export the final dataset

writer = pd.ExcelWriter('04sotu_with_gender_final.xlsx')
tweets_final.to_excel(writer,'Sheet1')
writer.save()